In [ ]:
import tempfile
import os
from pydub import AudioSegment

def compress_mp3_return_wav(original_audio_file, bitrate="128k"):
    audio = AudioSegment.from_file(original_audio_file, format="wav")

    # Temporary MP3 Conversion
    with tempfile.NamedTemporaryFile(suffix='.mp3') as temp_mp3:
        audio.export(temp_mp3.name, format="mp3", bitrate=bitrate)

        # Load back from the temporary MP3 file
        temp_audio = AudioSegment.from_mp3(temp_mp3.name)

        # Construct the output WAV filename
        original_filename = os.path.splitext(os.path.basename(original_audio_file))[0]
        output_folder = "BongoCompressed" + bitrate  # Dynamic output folder
        output_wav_filename = os.path.join(output_folder, original_filename + ".wav")

        # Create the output folder if it doesn't exist
        os.makedirs(output_folder, exist_ok=True)

        # Export and save as WAV
        temp_audio.export(output_wav_filename, format="wav")





In [ ]:
# Main Processing Logic
audio_folder = "Bongo"
bitrates = ["64k", "128k", "256k"]

for filename in os.listdir(audio_folder):
    if filename.endswith(".wav"):
        filepath = os.path.join(audio_folder, filename)
        for bitrate in bitrates:
            compress_mp3_return_wav(filepath, bitrate)

In [ ]:
import tempfile
import os
from pydub import AudioSegment

def compress_mp3_return_wav(original_audio_file, bitrate="128k"):
    audio = AudioSegment.from_file(original_audio_file, format="wav")

    # Temporary MP3 Conversion
    with tempfile.NamedTemporaryFile(suffix='.mp3') as temp_mp3:
        audio.export(temp_mp3.name, format="mp3", bitrate=bitrate)

        # Load back from the temporary MP3 file
        temp_audio = AudioSegment.from_mp3(temp_mp3.name)

        # Construct the output WAV filename
        original_filename = os.path.splitext(os.path.basename(original_audio_file))[0]
        output_folder = "AmapianoCompressed" + bitrate  # Dynamic output folder
        output_wav_filename = os.path.join(output_folder, original_filename + ".wav")

        # Create the output folder if it doesn't exist
        os.makedirs(output_folder, exist_ok=True)

        # Export and save as WAV
        temp_audio.export(output_wav_filename, format="wav")


In [ ]:
# Main Processing Logic
audio_folder = "Amapiano"
bitrates = ["64k", "128k", "256k"]

for filename in os.listdir(audio_folder):
    if filename.endswith(".wav"):
        filepath = os.path.join(audio_folder, filename)
        for bitrate in bitrates:
            compress_mp3_return_wav(filepath, bitrate)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files

!zip -r bongo64.zip BongoCompressed64k/

files.download('bongo64.zip')

In [ ]:
from google.colab import files

!zip -r bongo128.zip BongoCompressed128k/

files.download('bongo128.zip')

In [ ]:
from google.colab import files

!zip -r bongo256.zip BongoCompressed256k/

files.download('bongo256.zip')

In [ ]:
from google.colab import files

!zip -r amapiano64.zip AmapianoCompressed64k/

files.download('amapiano64.zip')

In [ ]:
from google.colab import files

!zip -r amapiano128.zip AmapianoCompressed128k/

files.download('amapiano128.zip')

In [ ]:
from google.colab import files

!zip -r amapiano256.zip AmapianoCompressed256k/

files.download('amapiano256.zip')

In [ ]:
import os
import csv
import soundfile as sf
import numpy as np

# Define the directories
directories = {
    '64': 'BongoCompressed64k',
    '128': 'BongoCompressed128k',
    '256': 'BongoCompressed256k'
}

# Open the CSV file
with open('Bongoresults.csv', 'w', newline='') as csvfile:
    fieldnames = ['filename', 'bitrate', 'snr', 'psnr', 'ssim']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    # Loop over the directories
    for bitrate, directory in directories.items():
        # Get all files in the directory
        files = os.listdir(directory)

        # Loop over the files
        for file in files:
            # Read the original and degraded audio files
            original_audio, fs = sf.read(f'Bongo/{file}')
            degraded_audio, fs = sf.read(f'{directory}/{file}')

            # Handle potential multi-channel audio
            if len(original_audio.shape) > 1:
                original_audio = original_audio[:, 0]  # Use the first channel
            if len(degraded_audio.shape) > 1:
                degraded_audio = degraded_audio[:, 0]  # Use the first channel

            # Calculate metrics
            snr_value = snr(original_audio, degraded_audio)
            psnr_value = psnr(original_audio, degraded_audio)
            ssim_value = ssim(original_audio, degraded_audio)

            # Write the results to the CSV file
            writer.writerow({
                'filename': file,
                'bitrate': bitrate,
                'snr': snr_value,
                'psnr': psnr_value,
                'ssim': ssim_value
            })